In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#Importing the dataset
db=pd.read_csv('credit_card_default_train.csv')
#Encoding the categorical variable
gender={"M":0 ,"F":1} 
status ={"Single":0 , "Other":1} 
study= {"Graduate":1 , "High School":2 , "Other":3} 
age= {"Less than 30":1 , "31-45":2, "46-65":3 , "More than 65":4}
db['AGE']=db['AGE'].map(age)
db['Gender']=db['Gender'].map(gender)
db['EDUCATION_STATUS']=db['EDUCATION_STATUS'].map(study)
db['MARITAL_STATUS']=db['MARITAL_STATUS'].map(status)
X=pd.get_dummies(db[['AGE','Balance_Limit_V1','Gender','EDUCATION_STATUS','MARITAL_STATUS','PAY_JULY','PAY_AUG','PAY_SEP','PAY_OCT','PAY_NOV','PAY_DEC']])
y=pd.get_dummies(db['NEXT_MONTH_DEFAULT'],drop_first=True)
X['score']=db['DUE_AMT_JULY']+db['DUE_AMT_AUG']+db['DUE_AMT_SEP']+db['DUE_AMT_OCT']+db['DUE_AMT_NOV']+db['DUE_AMT_DEC']-(db['PAID_AMT_JULY']+db['PAID_AMT_AUG']+db['PAID_AMT_SEP']+db['PAID_AMT_OCT']+db['PAID_AMT_NOV']+db['PAID_AMT_DEC'])


#Importing the testset
db=pd.read_csv('credit_card_default_test.csv')
#Encoding the categorical variable
gender={"M":0 ,"F":1} 
status ={"Single":0 , "Other":1} 
study= {"Graduate":1 , "High School":2 , "Other":3} 
age= {"Less than 30":1 , "31-45":2, "46-65":3 , "More than 65":4}
db['AGE']=db['AGE'].map(age)
db['Gender']=db['Gender'].map(gender)
db['EDUCATION_STATUS']=db['EDUCATION_STATUS'].map(study)
db['MARITAL_STATUS']=db['MARITAL_STATUS'].map(status)
X_test=pd.get_dummies(db[['AGE','Balance_Limit_V1','Gender','EDUCATION_STATUS','MARITAL_STATUS','PAY_JULY','PAY_AUG','PAY_SEP','PAY_OCT','PAY_NOV','PAY_DEC']])
X_test['score']=db['DUE_AMT_JULY']+db['DUE_AMT_AUG']+db['DUE_AMT_SEP']+db['DUE_AMT_OCT']+db['DUE_AMT_NOV']+db['DUE_AMT_DEC']-(db['PAID_AMT_JULY']+db['PAID_AMT_AUG']+db['PAID_AMT_SEP']+db['PAID_AMT_OCT']+db['PAID_AMT_NOV']+db['PAID_AMT_DEC'])



# Encoding the independent variable
from sklearn.preprocessing import OneHotEncoder


onehotencoder_0 = OneHotEncoder(categorical_features=[0])
X = onehotencoder_0.fit_transform(X).toarray()
X_test = onehotencoder_0.fit_transform(X_test).toarray()

#avoid dummy variable trap
X= X[:,1:]
X_test= X_test[:,1:]

onehotencoder_2 = OneHotEncoder(categorical_features=[4])
X = onehotencoder_2.fit_transform(X).toarray()
X_test = onehotencoder_2.fit_transform(X_test).toarray()

#avoid dummy variable trap
X= X[:,1:]
X_test= X_test[:,1:]


# Fitting XGBoost to the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X, y)
y_pred_1 = classifier.predict(X_test)



#Apply k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies1 = cross_val_score(estimator=classifier,X=X, y= y, cv=10)
accuracies1.mean()

# Building the optimal model using Backward Elimination
import statsmodels.api as sm
X = np.append(arr = np.ones((24000, 1)).astype(int), values = X, axis = 1)
X_test = np.append(arr = np.ones((6000, 1)).astype(int), values = X_test, axis = 1)
X_opt = X[:,:]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()


X_opt = X[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,19,20,21,22] ]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

X_opt = X[:, [0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,19,20,21,22] ]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()
#change the x_test as train set
X_test = X_test[:, [0,1,2,3,4,5,6,7,8,9,10,12,13,14,16,17,19,20,21,22] ]


# Fitting XGBoost to the optimize Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_opt, y)
y_pred_2 = classifier.predict(X_test)

#Apply k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies2 = cross_val_score(estimator=classifier,X=X_opt, y= y, cv=10)
accuracies2.mean()



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of in

0.8201666666666668